In [1]:
import os
import openeye.oechem as oechem
import qcportal as ptl
import qcelemental as qcel
import cmiles

## Explore the dataset

In [2]:
# view collections in QCArchive
client = ptl.FractalClient()
client.list_collections()

tagline
collection          name                                                
Dataset             GDB13-T                                  In progress
                    OpenFF Discrepancy Benchmark 1                  None
                    OpenFF NCI250K Boron 1                          None
                    OpenFF Optimization Set 1                       None
                    OpenFF VEHICLe Set 1                            None
...                                                                  ...
TorsionDriveDataset OpenFF Group1 Torsions                          None
                    OpenFF Primary TorsionDrive Benchmark 1         None
                    OpenFF Substituted Phenyl Set 1                 None
                    Pfizer Discrepancy Torsion Dataset 1            None
                    SMIRNOFF Coverage Torsion Set 1                 None

[70 rows x 1 columns]

In [3]:
# pull the collection of interest
ds = client.get_collection('OptimizationDataset', 'OpenFF Full Optimization Benchmark 1')

In [4]:
# view details about group type
ds.list_specifications()

,Description
Name,
default,Standard OpenFF optimization quantum chemistry...


In [5]:
spec = ds.list_specifications().index[0]
print(spec)

default


In [6]:
ds.list_specifications().iloc[0]['Description']

'Standard OpenFF optimization quantum chemistry specification.'

In [7]:
# see how many molecules in the set (conformers counted separately)
ds.status(spec)

,default
COMPLETE,26313
ERROR,420
INCOMPLETE,3


In [8]:
# show the number of optimization steps per entry
ds.counts().head(15)

,default
C[NH2+]C[C@@H](c1ccc(c(c1)O)O)O-0,36.0
C[NH2+]C[C@@H](c1ccc(c(c1)O)O)O-1,19.0
C[NH2+]C[C@@H](c1ccc(c(c1)O)O)O-2,25.0
C[NH2+]C[C@@H](c1ccc(c(c1)O)O)O-3,24.0
C[NH2+]C[C@@H](c1ccc(c(c1)O)O)O-4,28.0
C[NH2+]C[C@@H](c1ccc(c(c1)O)O)O-5,31.0
C[NH2+]C[C@@H](c1ccc(c(c1)O)O)O-6,23.0
C[NH2+]C[C@@H](c1ccc(c(c1)O)O)O-7,24.0
C[NH2+]C[C@@H](c1ccc(c(c1)O)O)O-8,35.0
C[NH2+]C[C@@H](c1ccc(c(c1)O)O)O-9,25.0


## Explore a molecule 

In [135]:
# check out a single molecule and its final geometry
optrec = ds.get_record(name="C[NH2+]C[C@@H](c1ccc(c(c1)O)O)O-0", specification="default")
print(optrec.status)
optrec.get_final_molecule()

RecordStatusEnum.complete


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

<Molecule(name='C9H14NO3' formula='C9H14NO3' hash='2cf0381')>

In [80]:
optrec.show_history()

In [113]:
optrec.get_final_energy()

-631.6931043923831

## Convert to OEMol, write to file (single mol) 

In [114]:
mol = optrec.get_final_molecule()
type(mol)

qcelemental.models.molecule.Molecule

In [115]:
# convert the qcelemental molecule to an OpenEye molecule
qcjson_mol = mol.dict(encoding='json')
oemol = cmiles.utils.load_molecule(qcjson_mol)
oemol

<oechem.OEMol; proxy of <Swig Object of type 'OEMolWrapper *' at 0x7f6d36265f30> >

In [116]:
# convert energy from Hartrees to kcal/mol
ene = optrec.get_final_energy()*qcel.constants.hartree2kcalmol

In [117]:
# add name and energy tag to the mol
oemol.SetTitle("full_1")
oechem.OESetSDData(oemol, "Energy QCArchive", str(ene))

True

In [118]:
# write molecule -- check that title and sd tag exists
ofs = oechem.oemolostream()
ofs.open("test.sdf")
oechem.OEWriteConstMolecule(ofs, oemol)

True

## Convert to OEMol, write to file, test conformer reading

In [219]:
# open an outstream file
outfile = 'test.sdf'
ofs = oechem.oemolostream()
#if os.path.exists(outfile):
#    raise FileExistsError("Output file {} already exists in {}".format(
#        outfile, os.getcwd()))
if not ofs.open(outfile):
    oechem.OEThrow.Fatal("Unable to open %s for writing" % outfile)

In [220]:
for i, index in enumerate(ds.df.index):
    
    # get the record of each entry
    record = ds.get_record(name=index, specification=spec)
    
    if i%10 == 0: 
        print(i)
    if i == 20:
        break
    print(index)
        
    if record.status == "COMPLETE":
    
        # get optimized molecule of the record
        qc_mol = record.get_final_molecule()

        # convert the qcelemental molecule to an OpenEye molecule
        qcjson_mol = qc_mol.dict(encoding='json')
        oemol = cmiles.utils.load_molecule(qcjson_mol)

        # convert energy from Hartrees to kcal/mol
        ene = record.get_final_energy()*qcel.constants.hartree2kcalmol

        # add name and energy tag to the mol
        oemol.SetTitle(f"full_{i+1}")
        oechem.OESetSDData(oemol, "Energy QCArchive", str(ene))
        
        # write molecule to file
        oechem.OEWriteConstMolecule(ofs, oemol)


0
C[NH2+]C[C@@H](c1ccc(c(c1)O)O)O-0
C[NH2+]C[C@@H](c1ccc(c(c1)O)O)O-1
C[NH2+]C[C@@H](c1ccc(c(c1)O)O)O-2
C[NH2+]C[C@@H](c1ccc(c(c1)O)O)O-3
C[NH2+]C[C@@H](c1ccc(c(c1)O)O)O-4
C[NH2+]C[C@@H](c1ccc(c(c1)O)O)O-5
C[NH2+]C[C@@H](c1ccc(c(c1)O)O)O-6
C[NH2+]C[C@@H](c1ccc(c(c1)O)O)O-7
C[NH2+]C[C@@H](c1ccc(c(c1)O)O)O-8
C[NH2+]C[C@@H](c1ccc(c(c1)O)O)O-9
10
CO/N=C/1\C[N@](C[C@H]1C[NH3+])c2c(cc3c(=O)c(cn(c3n2)C4CC4)C(=O)[O-])F-0
CO/N=C/1\C[N@](C[C@H]1C[NH3+])c2c(cc3c(=O)c(cn(c3n2)C4CC4)C(=O)[O-])F-1
CO/N=C/1\C[N@](C[C@H]1C[NH3+])c2c(cc3c(=O)c(cn(c3n2)C4CC4)C(=O)[O-])F-2
CO/N=C/1\C[N@](C[C@H]1C[NH3+])c2c(cc3c(=O)c(cn(c3n2)C4CC4)C(=O)[O-])F-3
CO/N=C/1\C[N@](C[C@H]1C[NH3+])c2c(cc3c(=O)c(cn(c3n2)C4CC4)C(=O)[O-])F-4
CO/N=C/1\C[N@](C[C@H]1C[NH3+])c2c(cc3c(=O)c(cn(c3n2)C4CC4)C(=O)[O-])F-5
CO/N=C/1\C[N@](C[C@H]1C[NH3+])c2c(cc3c(=O)c(cn(c3n2)C4CC4)C(=O)[O-])F-6
c1cc(ccc1[C@H]2C[NH2+]CCc3c2cc(c(c3Cl)O)O)O-0
c1cc(ccc1[C@H]2C[NH2+]CCc3c2cc(c(c3Cl)O)O)O-1
c1cc(ccc1[C@H]2C[NH2+]CCc3c2cc(c(c3Cl)O)O)O-2
20


In [221]:
ofs.close()

In [230]:
#https://docs.eyesopen.com/toolkits/python/oechemtk/oemol.html
ifs = oechem.oemolistream()
#ifs.SetConfTest(oechem.OEAbsCanonicalConfTest())
ifs.SetConfTest(oechem.OEAbsoluteConfTest(False)) # false means confs may have diff titles
if not ifs.open(outfile):
    raise FileNotFoundError(f"Unable to open {outfile} for reading")
mols = ifs.GetOEMols()

In [231]:
for i, mol in enumerate(mols):
    for j, conf in enumerate(mol.GetConfs()):
        print(i, mol.NumConfs(), conf.GetTitle(), oechem.OEMolToSmiles(conf))

0 10 full_1 C[NH2+]CC(c1ccc(c(c1)O)O)O
0 10 full_2 C[NH2+]CC(c1ccc(c(c1)O)O)O
0 10 full_3 C[NH2+]CC(c1ccc(c(c1)O)O)O
0 10 full_4 C[NH2+]CC(c1ccc(c(c1)O)O)O
0 10 full_5 C[NH2+]CC(c1ccc(c(c1)O)O)O
0 10 full_6 C[NH2+]CC(c1ccc(c(c1)O)O)O
0 10 full_7 C[NH2+]CC(c1ccc(c(c1)O)O)O
0 10 full_8 C[NH2+]CC(c1ccc(c(c1)O)O)O
0 10 full_9 C[NH2+]CC(c1ccc(c(c1)O)O)O
0 10 full_10 C[NH2+]CC(c1ccc(c(c1)O)O)O
1 1 full_14 CON=C1C[NH+](CC1CN)c2c(cc3c(=O)c(cn(c3n2)C4CC4)C(=O)[O-])F
2 1 full_16 CON=C1CN(CC1C[NH3+])c2c(cc3c(=O)c(cn(c3n2)C4CC4)C(=O)[O-])F
3 1 full_17 CON=C1CN(CC1CN)c2c(cc3c(=O)c(cn(c3n2)C4CC4)C(=O)O)F
4 3 full_18 c1cc(ccc1C2C[NH2+]CCc3c2cc(c(c3Cl)O)O)O
4 3 full_19 c1cc(ccc1C2C[NH2+]CCc3c2cc(c(c3Cl)O)O)O
4 3 full_20 c1cc(ccc1C2C[NH2+]CCc3c2cc(c(c3Cl)O)O)O


## Look into inconsistent SMILES strings after conversion

In [229]:
# check out a single molecule and its final geometry
optrec = ds.get_record(name="CO/N=C/1\C[N@](C[C@H]1C[NH3+])c2c(cc3c(=O)c(cn(c3n2)C4CC4)C(=O)[O-])F-3", specification="default")
print(optrec.status)
optrec.get_final_molecule()

RecordStatusEnum.complete


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

<Molecule(name='C18FH20N5O4' formula='C18FH20N5O4' hash='ba9673c')>

In [228]:
# check out a single molecule and its final geometry
optrec = ds.get_record(name="CO/N=C/1\C[N@](C[C@H]1C[NH3+])c2c(cc3c(=O)c(cn(c3n2)C4CC4)C(=O)[O-])F-6", specification="default")
print(optrec.status)
optrec.get_final_molecule()

RecordStatusEnum.complete


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

<Molecule(name='C18FH20N5O4' formula='C18FH20N5O4' hash='550ce40')>

## Write out the whole set

This took about 5.5 hours

In [11]:
# open an outstream file
outfile = 'whole.sdf'
ofs = oechem.oemolostream()
#if os.path.exists(outfile):
#    raise FileExistsError("Output file {} already exists in {}".format(
#        outfile, os.getcwd()))
if not ofs.open(outfile):
    oechem.OEThrow.Fatal("Unable to open %s for writing" % outfile)

In [12]:
for i, index in enumerate(ds.df.index):
    
    # get the record of each entry
    record = ds.get_record(name=index, specification=spec)
    
    if i%100 == 0: 
        print(i)
        
    if record.status == "COMPLETE":
    
        # get optimized molecule of the record
        qc_mol = record.get_final_molecule()

        # convert the qcelemental molecule to an OpenEye molecule
        qcjson_mol = qc_mol.dict(encoding='json')
        oemol = cmiles.utils.load_molecule(qcjson_mol)

        # convert energy from Hartrees to kcal/mol
        ene = record.get_final_energy()*qcel.constants.hartree2kcalmol

        # add name and energy tag to the mol
        oemol.SetTitle(f"full_{i+1}")
        oechem.OESetSDData(oemol, "SMILES QCArchive", index)
        oechem.OESetSDData(oemol, "Energy QCArchive", str(ene))
        
        # write molecule to file
        oechem.OEWriteConstMolecule(ofs, oemol)

ofs.close()

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18